<a href="https://colab.research.google.com/github/GEOFFREY-MO/My-data/blob/main/tomorrow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import necessary libraries
import pandas as pd
import requests
from datetime import datetime, timedelta
import os

**Making Requests for the API keys**

In [ ]:
url = "https://api.tomorrow.io/v4/timelines"

querystring = {
    'location':'Kenya',
    'fields':['temperature','cloudCover','humidity','pressureSurfaceLevel','rainIntensity'],
    'units':'imperial',
    'timesteps':'1d',
    'apikey':'U188WPIa9pIzejPlPBU5tDd6kU8WzIrF'}

In [ ]:
response = requests.request('GET',url,params = querystring)
print(response.text)

{"data":{"timelines":[{"timestep":"1d","endTime":"2023-11-20T03:00:00Z","startTime":"2023-11-15T03:00:00Z","intervals":[{"startTime":"2023-11-15T03:00:00Z","values":{"cloudCover":100,"humidity":98.58,"pressureSurfaceLevel":28.83,"rainIntensity":0.05,"temperature":86.89}},{"startTime":"2023-11-16T03:00:00Z","values":{"cloudCover":100,"humidity":95.84,"pressureSurfaceLevel":28.84,"rainIntensity":0,"temperature":87.59}},{"startTime":"2023-11-17T03:00:00Z","values":{"cloudCover":100,"humidity":93.91,"pressureSurfaceLevel":28.86,"rainIntensity":0.04,"temperature":86.14}},{"startTime":"2023-11-18T03:00:00Z","values":{"cloudCover":100,"humidity":88.53,"pressureSurfaceLevel":28.86,"rainIntensity":0.03,"temperature":86.31}},{"startTime":"2023-11-19T03:00:00Z","values":{"cloudCover":91.59,"humidity":81.73,"pressureSurfaceLevel":28.78,"rainIntensity":0.04,"temperature":88.83}},{"startTime":"2023-11-20T03:00:00Z","values":{"cloudCover":98.73,"humidity":81.73,"pressureSurfaceLevel":28.77,"rainInten

In [ ]:
#identifying the type of the data
type(response)

requests.models.Response

In [ ]:
tmpr=[]
hmd=[]
dat=[]
RI=[]
CV=[]
ps=[]
results = response.json()['data']['timelines'][0]['intervals']
for daily_result in results:
   date = daily_result['startTime'][0:10]
   temp = round(daily_result['values']['temperature'])
   hum=daily_result['values']['humidity']
   cloud=daily_result['values']['cloudCover']
   press=daily_result['values']['pressureSurfaceLevel']
   rain=daily_result['values']['rainIntensity']
   dat.append(date)
   tmpr.append(temp)
   hmd.append(hum)
   RI.append(rain)
   CV.append(cloud)
   ps.append(press)

In [ ]:
data = pd.DataFrame(list(zip(dat,hmd,tmpr,RI,CV,ps)),columns= ['Date','Humidity','Temperature','Rainfall Intensity','CloudCover','Pressure'])
data.head()

,Date,Humidity,Temperature,Rainfall Intensity,CloudCover,Pressure
0,2023-11-15,98.58,87,0.05,100.00,28.83
1,2023-11-16,95.84,88,0.00,100.00,28.84
2,2023-11-17,93.91,86,0.04,100.00,28.86
3,2023-11-18,88.53,86,0.03,100.00,28.86
4,2023-11-19,81.73,89,0.04,91.59,28.78


**Creating a function to save data to csv after every 7 day**

In [ ]:
from numpy.core.multiarray import datetime_data
#create a function that saves data to a csv
def save_to_csv(data):
  file_path = 'weather_data.csv'
  #check if the file already exists
  if os.path.exists(file_path):
    #Load existing data
    existing_data = pd.read_csv(file_path)
    #append new data
    data = pd.concat([existing_data,data], ignore_index=True)
  #save data to csv
  data.to_csv(file_path, index=False)

#Scheduling the script for running every 7 days
current_date = datetime.now()
previous_run_date = current_date -timedelta(days= 7)

#compare with the last run date
if current_date.date()>=previous_run_date.date():
  #save data to csv
  save_to_csv(data)

